### 은닉 마르코프 모형 (HMM) - 기초

강의 슬라이드의 HMM 모형을 만들어 본다. 

![](https://drive.google.com/uc?export=view&id=1yJcAnq6u2-lyGXsMvgEJxH9J8Ek0IrPf)

In [1]:
# 다음 라이브러리를 설치한다.
!pip install pomegranate

     |████████████████████████████████| 17.9 MB 131 kB/s 
     |████████████████████████████████| 15.7 MB 77 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
# 라이브러리를 불러온다.
import numpy as np
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

ValueError: ignored

#### 1. 모델 파라미터 설정:

1.1. 상태의 초기 확률 (Initial Probabilities):

|        | Prob |
|: --- :|: --- :|
| $Sunny$  | 0.50 |
|$Rainny$  | 0.50 |

1.2. 전이 확률 (Transition Probabilities):

|   | $Sunny$ | $Rainny$ |
|:---:|:---:|:---:|
|$Sunny$| 0.80 | 0.40 |
|$Rainny$| 0.20 | 0.60 | 

1.3. 출력 확률 (Emission Probabilities):

|    |  $Sunny$  | $Rainny$ |
|: ---: |: --- :|: ---: |
| $Game$ |   0.10  | 0.80 |
| $Fitness$ | 0.90 | 0.20 |


#### 2. 모델 생성:

In [ ]:
# 모델 생성.
my_hmm = HiddenMarkovModel(name="my_model")

In [ ]:
# 상태와 출력확률 P(O | S) 설정.
Sunny_emissions = DiscreteDistribution({"Game": 0.1, "Fitness": 0.9})
Sunny_state = State(Sunny_emissions, name="Sunny_state")

Rainny_emissions = DiscreteDistribution({"Game": 0.8, "Fitness": 0.2})
Rainny_state = State(Rainny_emissions, name="Rainny_state")

# 모델에 상태 추가.
my_hmm.add_states(Sunny_state)
my_hmm.add_states(Rainny_state)

In [ ]:
# 초기 확률 설정.
my_hmm.add_transition(my_hmm.start, Sunny_state, 0.5)
my_hmm.add_transition(my_hmm.start, Rainny_state, 0.5)

In [ ]:
# 전이 확률 P(S'|S) 설정.
my_hmm.add_transition(Sunny_state, Sunny_state, 0.8)  # 80% Sunny -> Sunny
my_hmm.add_transition(Sunny_state, Rainny_state, 0.2)  # 20% Sunny -> Rainny

my_hmm.add_transition(Rainny_state, Sunny_state, 0.4)  # 40% Rainny -> Sunny
my_hmm.add_transition(Rainny_state, Rainny_state, 0.6)  # 60% Rainny -> Sunny

In [ ]:
# 모델 최종 완성.
my_hmm.bake()

In [ ]:
# 모델의 구조 출력.
print("Edge counts: ",my_hmm.edge_count()) 
print("Node counts: ",my_hmm.node_count()) 

In [ ]:
# 입력된 상태의 이름 확인.
for s in my_hmm.states:
    print(s.name)

#### 3. 모델을 사용한 예측 (Inference):

In [ ]:
# 다음과 같은 시퀀스가 관찰 되었다 전제한다.
observations = ["Game", "Fitness", "Game"]

In [ ]:
# Viterbi 알고리즘으로 가장 우도 (likelihood)가 높은 경로를 계산한다.
viterbi_likelihood, viterbi_path = my_hmm.viterbi(observations)

In [ ]:
# The likely weather.
weather_pred = []
for s in viterbi_path[1:]:
    weather_pred.append(s[1].name)
print("제일 유력한 날씨 : {}".format(weather_pred))
print("우도 함수의 값   : {:0.4f}".format(np.exp(viterbi_likelihood)))